In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import torch
import tqdm
import h5py
import pandas as pd
import tables

In [5]:
# the "data" containing too much signal
features=pd.read_hdf("events_anomalydetection_v2.features.h5")

# additionally produced bkg
features_extrabkg = pd.read_hdf("events_anomalydetection_qcd_extra_inneronly_features.h5")

In [6]:
## to be split among the different sets 
features_extrabkg1 = features_extrabkg[:312858]

## to be used to enhance the evalaution
features_extrabkg2 = features_extrabkg[312858:]

features_sig=features[features['label']==1]
features_bg=features[features['label']==0]

In [7]:
print(np.array(features_bg[['mj1','mj2']]).shape)

(1000000, 2)


In [8]:
print(features_bg.keys)

<bound method NDFrame.keys of                pxj1         pyj1         pzj1         mj1      tau1j1  \
0      -1467.239990   611.502014   511.101990   38.896000    8.290660   
1      -1211.239990   347.315002   547.963013  389.532013  191.804001   
2      -1229.619995   649.857971     8.089170   72.155502   47.168098   
3       -693.304016 -1046.729980  1716.910034   55.797798   24.788601   
4      -1488.199951   -25.370100   -30.989700   84.891502   26.878799   
...             ...          ...          ...         ...         ...   
999995  -286.550995 -1310.829956 -1510.910034  147.516998   60.997799   
999996   918.562988   951.195984 -1622.569946   32.242199    5.894100   
999997  1447.219971  -547.710999   827.945007  396.112000  181.406998   
999998   200.035995 -1252.869995    27.924900  363.790985  139.281998   
999999  -811.379028  1050.810059     5.019200  150.214996   92.509003   

            tau2j1     tau3j1         pxj2         pyj2         pzj2  \
0         4.836080   

In [9]:
# Read from data
jet1_bg = np.array(features_bg[['pxj1', 'pyj1','pzj1']])
jet2_bg = np.array(features_bg[['pxj2','pyj2','pzj2']])
mj1mj2_bg = np.array(features_bg[['mj1','mj2']])
tau21_bg = np.array(features_bg[['tau2j1','tau2j2']])/(1e-5+np.array(features_bg[['tau1j1','tau1j2']]))
tau32_bg = np.array(features_bg[['tau3j1','tau3j2']])/(1e-5+np.array(features_bg[['tau2j1','tau2j2']]))

jet1_sig = np.array(features_sig[['pxj1', 'pyj1','pzj1']])
jet2_sig = np.array(features_sig[['pxj2','pyj2','pzj2']])
mj1mj2_sig = np.array(features_sig[['mj1','mj2']])
tau21_sig = np.array(features_sig[['tau2j1','tau2j2']])/(1e-5+np.array(features_sig[['tau1j1','tau1j2']]))
tau32_sig = np.array(features_sig[['tau3j1','tau3j2']])/(1e-5+np.array(features_sig[['tau2j1','tau2j2']]))


jet1_extrabkg1 = np.array(features_extrabkg1[['pxj1', 'pyj1','pzj1']])
jet2_extrabkg1 = np.array(features_extrabkg1[['pxj2','pyj2','pzj2']])
mj1mj2_extrabkg1 = np.array(features_extrabkg1[['mj1','mj2']])
tau21_extrabkg1 = np.array(features_extrabkg1[['tau2j1','tau2j2']])/(1e-5+np.array(features_extrabkg1[['tau1j1','tau1j2']]))
tau32_extrabkg1 = np.array(features_extrabkg1[['tau3j1','tau3j2']])/(1e-5+np.array(features_extrabkg1[['tau2j1','tau2j2']]))

jet1_extrabkg2 = np.array(features_extrabkg2[['pxj1', 'pyj1','pzj1']])
jet2_extrabkg2 = np.array(features_extrabkg2[['pxj2','pyj2','pzj2']])
mj1mj2_extrabkg2 = np.array(features_extrabkg2[['mj1','mj2']])
tau21_extrabkg2 = np.array(features_extrabkg2[['tau2j1','tau2j2']])/(1e-5+np.array(features_extrabkg2[['tau1j1','tau1j2']]))
tau32_extrabkg2 = np.array(features_extrabkg2[['tau3j1','tau3j2']])/(1e-5+np.array(features_extrabkg2[['tau2j1','tau2j2']]))

In [10]:
# Sorting of mj1 and mj2:
# Identifies which column has the minimum of mj1 and mj2, and sorts it so the new array mjmin contains the 
# mj with the smallest energy, and mjmax is the one with the biggest.
mjmin_bg = mj1mj2_bg[range(len(mj1mj2_bg)), np.argmin(mj1mj2_bg, axis=1)] 
mjmax_bg = mj1mj2_bg[range(len(mj1mj2_bg)), np.argmax(mj1mj2_bg, axis=1)]
mjmin_sig = mj1mj2_sig[range(len(mj1mj2_sig)), np.argmin(mj1mj2_sig, axis=1)]
mjmax_sig = mj1mj2_sig[range(len(mj1mj2_sig)), np.argmax(mj1mj2_sig, axis=1)]
mjmin_extrabkg1 = mj1mj2_extrabkg1[range(len(mj1mj2_extrabkg1)), np.argmin(mj1mj2_extrabkg1, axis=1)] 
mjmax_extrabkg1 = mj1mj2_extrabkg1[range(len(mj1mj2_extrabkg1)), np.argmax(mj1mj2_extrabkg1, axis=1)]
mjmin_extrabkg2 = mj1mj2_extrabkg2[range(len(mj1mj2_extrabkg2)), np.argmin(mj1mj2_extrabkg2, axis=1)] 
mjmax_extrabkg2 = mj1mj2_extrabkg2[range(len(mj1mj2_extrabkg2)), np.argmax(mj1mj2_extrabkg2, axis=1)]


In [11]:
# Then we do the same sorting for the taus
tau21min_bg=tau21_bg[range(len(mj1mj2_bg)), np.argmin(mj1mj2_bg, axis=1)]
tau21max_bg=tau21_bg[range(len(mj1mj2_bg)), np.argmax(mj1mj2_bg, axis=1)]
tau21min_sig=tau21_sig[range(len(mj1mj2_sig)), np.argmin(mj1mj2_sig, axis=1)]
tau21max_sig=tau21_sig[range(len(mj1mj2_sig)), np.argmax(mj1mj2_sig, axis=1)]
tau21min_extrabkg1 = tau21_extrabkg1[range(len(mj1mj2_extrabkg1)), np.argmin(mj1mj2_extrabkg1, axis=1)]
tau21max_extrabkg1 = tau21_extrabkg1[range(len(mj1mj2_extrabkg1)), np.argmax(mj1mj2_extrabkg1, axis=1)]
tau21min_extrabkg2 = tau21_extrabkg2[range(len(mj1mj2_extrabkg2)), np.argmin(mj1mj2_extrabkg2, axis=1)]
tau21max_extrabkg2 = tau21_extrabkg2[range(len(mj1mj2_extrabkg2)), np.argmax(mj1mj2_extrabkg2, axis=1)]

tau32min_bg=tau32_bg[range(len(mj1mj2_bg)), np.argmin(mj1mj2_bg, axis=1)]
tau32max_bg=tau32_bg[range(len(mj1mj2_bg)), np.argmax(mj1mj2_bg, axis=1)]
tau32min_sig=tau32_sig[range(len(mj1mj2_sig)), np.argmin(mj1mj2_sig, axis=1)]
tau32max_sig=tau32_sig[range(len(mj1mj2_sig)), np.argmax(mj1mj2_sig, axis=1)]
tau32min_extrabkg1 = tau32_extrabkg1[range(len(mj1mj2_extrabkg1)), np.argmin(mj1mj2_extrabkg1, axis=1)]
tau32max_extrabkg1 = tau32_extrabkg1[range(len(mj1mj2_extrabkg1)), np.argmax(mj1mj2_extrabkg1, axis=1)]
tau32min_extrabkg2 = tau32_extrabkg2[range(len(mj1mj2_extrabkg2)), np.argmin(mj1mj2_extrabkg2, axis=1)]
tau32max_extrabkg2 = tau32_extrabkg2[range(len(mj1mj2_extrabkg2)), np.argmax(mj1mj2_extrabkg2, axis=1)]

In [12]:
# Calculate mjj and collect the features into a dataset, plus mark signal/bg with 1/0
pjj_sig = (np.array(features_sig[['pxj1','pyj1','pzj1']])+np.array(features_sig[['pxj2','pyj2','pzj2']]))
Ejj_sig = np.sqrt(np.sum(np.array(features_sig[['pxj1','pyj1','pzj1','mj1']])**2, axis=1))\
    +np.sqrt(np.sum(np.array(features_sig[['pxj2','pyj2','pzj2','mj2']])**2, axis=1))
mjj_sig = np.sqrt(Ejj_sig**2-np.sum(pjj_sig**2, axis=1))

pjj_bg = (np.array(features_bg[['pxj1','pyj1','pzj1']])+np.array(features_bg[['pxj2','pyj2','pzj2']]))
Ejj_bg = np.sqrt(np.sum(np.array(features_bg[['pxj1','pyj1','pzj1','mj1']])**2, axis=1))\
    +np.sqrt(np.sum(np.array(features_bg[['pxj2','pyj2','pzj2','mj2']])**2, axis=1))
mjj_bg = np.sqrt(Ejj_bg**2-np.sum(pjj_bg**2, axis=1))

pjj_extrabkg1 = (np.array(features_extrabkg1[['pxj1','pyj1','pzj1']])+np.array(features_extrabkg1[['pxj2','pyj2','pzj2']]))
Ejj_extrabkg1 = np.sqrt(np.sum(np.array(features_extrabkg1[['pxj1','pyj1','pzj1','mj1']])**2, axis=1))\
    +np.sqrt(np.sum(np.array(features_extrabkg1[['pxj2','pyj2','pzj2','mj2']])**2, axis=1))
mjj_extrabkg1 = np.sqrt(Ejj_extrabkg1**2-np.sum(pjj_extrabkg1**2, axis=1))
pjj_extrabkg2 = (np.array(features_extrabkg2[['pxj1','pyj1','pzj1']])+np.array(features_extrabkg2[['pxj2','pyj2','pzj2']]))
Ejj_extrabkg2 = np.sqrt(np.sum(np.array(features_extrabkg2[['pxj1','pyj1','pzj1','mj1']])**2, axis=1))\
    +np.sqrt(np.sum(np.array(features_extrabkg2[['pxj2','pyj2','pzj2','mj2']])**2, axis=1))
mjj_extrabkg2 = np.sqrt(Ejj_extrabkg2**2-np.sum(pjj_extrabkg2**2, axis=1))

# ORDER OF THINGS HERE

In [17]:
print(mjj_bg.shape)
print(jet1_bg.shape)

dataset_bg = np.dstack((mjj_bg/1000,  mjmin_bg/1000, (mjmax_bg-mjmin_bg)/1000, tau21min_bg, tau21max_bg, tau32min_bg, tau32max_bg, np.zeros(len(mjj_bg))))
print(dataset_bg.shape)

(1000000,)
(1000000, 3)
(1, 1000000, 8)


In [22]:
##ORDER OF THINGS
dataset_bg = np.dstack((mjj_bg/1000, mjmin_bg/1000, (mjmax_bg-mjmin_bg)/1000, tau21min_bg, tau21max_bg, tau32min_bg, tau32max_bg, np.zeros(len(mjj_bg))))[0]
dataset_bg = np.hstack((jet1_bg, jet2_bg, dataset_bg))

dataset_sig = np.dstack((mjj_sig/1000, mjmin_sig/1000, (mjmax_sig-mjmin_sig)/1000, tau21min_sig, tau21max_sig, tau32min_sig, tau32max_sig, np.ones(len(mjj_sig))))[0]
dataset_sig = np.hstack((jet1_sig, jet2_sig, dataset_sig))

dataset_extrabkg1 = np.dstack((mjj_extrabkg1/1000, mjmin_extrabkg1/1000, (mjmax_extrabkg1-mjmin_extrabkg1)/1000, tau21min_extrabkg1, tau21max_extrabkg1, tau32min_extrabkg1, tau32max_extrabkg1, np.zeros(len(mjj_extrabkg1))))[0]
dataset_extrabkg1 = np.hstack((jet1_extrabkg1, jet2_extrabkg1, dataset_extrabkg1))

dataset_extrabkg2 = np.dstack((mjj_extrabkg2/1000, mjmin_extrabkg2/1000, (mjmax_extrabkg2-mjmin_extrabkg2)/1000, tau21min_extrabkg2, tau21max_extrabkg2, tau32min_extrabkg2, tau32max_extrabkg2, np.zeros(len(mjj_extrabkg2))))[0]
dataset_extrabkg2 = np.hstack((jet1_extrabkg2, jet2_extrabkg2, dataset_extrabkg2))

In [23]:
#np.random.seed(args.seed) # Set the random seed so we get a deterministic result

np.random.shuffle(dataset_sig)

n_sig = 1000


In [24]:
data_all = np.concatenate((dataset_bg, dataset_sig[:n_sig]))
indices = np.array(range(len(data_all))).astype('int')
np.random.shuffle(indices)
data_all = data_all[indices]
indices_extrabkg1 = np.array(range(len(dataset_extrabkg1))).astype('int')
np.random.shuffle(indices_extrabkg1)
dataset_extrabkg1 = dataset_extrabkg1[indices_extrabkg1]
indices_extrabkg2 = np.array(range(len(dataset_extrabkg2))).astype('int')
np.random.shuffle(indices_extrabkg2)
dataset_extrabkg2 = dataset_extrabkg2[indices_extrabkg2]

In [25]:
# format of data_all: mjj (TeV), mjmin (TeV), mjmax-mjmin (TeV), tau21(mjmin), tau21 (mjmax), tau32(mjmin), tau32 (mjmax), sigorbg label

minmass=3.3
maxmass=3.7

innermask = (data_all[:,0]>minmass) & (data_all[:,0]<maxmass)
outermask = ~innermask
innerdata = data_all[innermask]
outerdata = data_all[outermask]

innermask_extrabkg1 = (dataset_extrabkg1[:,0]>minmass) & (dataset_extrabkg1[:,0]<maxmass)
innerdata_extrabkg1 = dataset_extrabkg1[innermask_extrabkg1]
innermask_extrabkg2 = (dataset_extrabkg2[:,0]>minmass) & (dataset_extrabkg2[:,0]<maxmass)
innerdata_extrabkg2 = dataset_extrabkg2[innermask_extrabkg2]

In [26]:
outerdata_train = outerdata[:500000]
outerdata_val = outerdata[500000:]

innerdata_train = innerdata[:60000]
innerdata_val = innerdata[60000:120000]

innerdata_extrasig = dataset_sig[n_sig:]
innerdata_extrasig = innerdata_extrasig[(innerdata_extrasig[:,0]>minmass) & (innerdata_extrasig[:,0]<maxmass)]

## splitting extra signal into train, val and test set
n_sig_test = 20000
n_extrasig_train =  (innerdata_extrasig.shape[0]-n_sig_test)//2
innerdata_extrasig_test = innerdata_extrasig[:n_sig_test]
innerdata_extrasig_train = innerdata_extrasig[n_sig_test:n_sig_test+n_extrasig_train]
innerdata_extrasig_val = innerdata_extrasig[n_sig_test+n_extrasig_train:]

## splitting extra bkg (1) into train, val and test set
n_bkg_test = 40000
n_extrabkg_train =  (innerdata_extrabkg1.shape[0]-n_bkg_test)//2
innerdata_extrabkg1_test = innerdata_extrabkg1[:n_bkg_test]
innerdata_extrabkg1_train = innerdata_extrabkg1[n_bkg_test:n_bkg_test+n_extrabkg_train]
innerdata_extrabkg1_val = innerdata_extrabkg1[n_bkg_test+n_extrabkg_train:]

## putting together artificial test set
innerdata_test = np.vstack((innerdata_extrabkg1_test, innerdata_extrasig_test))

In [27]:
np.save('outerdata_train_6var.npy', outerdata_train)
np.save('outerdata_test_6var.npy', outerdata_val)
np.save('innerdata_train_6var.npy', innerdata_train)
np.save('innerdata_val_6var.npy', innerdata_val)   
np.save('innerdata_test_6var.npy', innerdata_test)      
np.save('innerdata_extrasig_train_6var.npy', innerdata_extrasig_train)
np.save('innerdata_extrasig_val_6var.npy', innerdata_extrasig_val)
np.save('innerdata_extrabkg_train_6var.npy', innerdata_extrabkg1_train)
np.save('innerdata_extrabkg_val_6var.npy', innerdata_extrabkg1_val)
np.save('innerdata_extrabkg_test_6var.npy', innerdata_extrabkg2)

# Old

In [9]:
print(f1['df'].keys())

<KeysViewHDF5 ['axis0', 'axis1', 'block0_items', 'block0_values']>


In [10]:
for key in ["axis0", 'axis1' , 'block0_items', 'block0_values']:
    print(f1['df'][key])

<HDF5 dataset "axis0": shape (15,), type "|S6">
<HDF5 dataset "axis1": shape (1100000,), type "<i8">
<HDF5 dataset "block0_items": shape (15,), type "|S6">
<HDF5 dataset "block0_values": shape (1100000, 15), type "<f8">


In [11]:
for i in range(15):
    print(f1['df']['axis0'][i])

b'pxj1'
b'pyj1'
b'pzj1'
b'mj1'
b'tau1j1'
b'tau2j1'
b'tau3j1'
b'pxj2'
b'pyj2'
b'pzj2'
b'mj2'
b'tau1j2'
b'tau2j2'
b'tau3j2'
b'label'


In [11]:
for i in range(15):
    print(f1['df']['block0_items'][i])

b'pxj1'
b'pyj1'
b'pzj1'
b'mj1'
b'tau1j1'
b'tau2j1'
b'tau3j1'
b'pxj2'
b'pyj2'
b'pzj2'
b'mj2'
b'tau1j2'
b'tau2j2'
b'tau3j2'
b'label'


In [6]:
df_test = pandas.read_hdf("events_anomalydetection.h5",stop=10000)
print(df_test.shape)
print("Memory in GB:",sum(df_test.memory_usage(deep=True)) / (1024**3))


(10000, 2101)
Memory in GB: 0.15661120414733887
